In [1]:
2pip install torch

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install torchvision

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Looking in indexes: https://download.pytorch.org/whl/cu118


In [5]:
pip install timm

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/48.4 kB ? eta -:--:--
     -------- ------------------------------- 10.2/48.4 kB ? eta -:--:--
     -------------------------------- ----- 41.0/48.4 kB 393.8 kB/s eta 0:00:01
     -------------------------------------- 48.4/48.4 kB 349.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   -- ------------------------------------- 0.1/2.3 MB 4.3 MB/s eta 0:00:01
   ----------- ---------------------------- 0.7/2.3 MB 10.6 MB/s eta 0:00:01
   ------------------------- -------------- 1.5/2.3 MB 11.6 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 14.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torch import nn, optim
from timm import create_model
from tqdm import tqdm


In [13]:
# Define transformations for data augmentation and normalization
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images for ViT
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize using ImageNet stats
])

# Load training data
train_data_path = '/Users/anishvirkhare/Downloads/archive/train' 
train_data = datasets.ImageFolder(root=train_data_path, transform=transform)

# Split into training and validation sets
train_size = int(0.8 * len(train_data))
val_size = len(train_data) - train_size
train_dataset, val_dataset = random_split(train_data, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Optional: Check dataset sizes
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")


Training samples: 12556
Validation samples: 3140


In [14]:
# Create a Vision Transformer model with 23 classes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = create_model('vit_base_patch16_224', pretrained=True, num_classes=23).to(device)

In [15]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=3e-4)

In [17]:
# Training loop
epochs = 30    # Number of epochs
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    # Training
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} - Training"):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track training loss and accuracy
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

    avg_train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train

    # Validation
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} - Validation"):
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Track validation accuracy
            _, predicted = torch.max(outputs, 1)
            correct_val += (predicted == labels).sum().item()
            total_val += labels.size(0)

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * correct_val / total_val

    # Print epoch summary
    print(f"Epoch {epoch+1}/{epochs} Summary:")
    print(f"  Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%")
    print(f"  Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")


Epoch 1/30 - Validation: 100%|█████████████████████████████████████████████████████████| 99/99 [00:51<00:00,  1.93it/s]


Epoch 1/30 Summary:
  Training Loss: 2.8796, Training Accuracy: 15.40%
  Validation Loss: 2.7370, Validation Accuracy: 18.89%


Epoch 2/30 - Validation: 100%|█████████████████████████████████████████████████████████| 99/99 [00:51<00:00,  1.91it/s]


Epoch 2/30 Summary:
  Training Loss: 2.6959, Training Accuracy: 20.04%
  Validation Loss: 2.6950, Validation Accuracy: 21.02%


Epoch 3/30 - Validation: 100%|█████████████████████████████████████████████████████████| 99/99 [00:51<00:00,  1.92it/s]


...

Epoch 28/30 Summary:
  Training Loss: 0.2851, Training Accuracy: 90.58%
  Validation Loss: 3.5632, Validation Accuracy: 34.97%


Epoch 29/30 - Validation: 100%|████████████████████████████████████████████████████████| 99/99 [00:48<00:00,  2.02it/s]


Epoch 29/30 Summary:
  Training Loss: 0.2528, Training Accuracy: 91.69%
  Validation Loss: 3.8626, Validation Accuracy: 35.32%


Epoch 30/30 - Validation: 100%|████████████████████████████████████████████████████████| 99/99 [00:48<00:00,  2.04it/s]

Epoch 30/30 Summary:
  Training Loss: 0.9619, Training Accuracy: 73.58%
  Validation Loss: 3.3663, Validation Accuracy: 9.01%


In [18]:
torch.save(model, "C:\\Users\\cl502_25\\Downloads\\vit.pth")

### ViT test

In [7]:
model_path = '/Users/anishvirkhare/Downloads/vit.pth'

# Load the complete model
model = torch.load(model_path, map_location=torch.device('cpu'))

device = torch.device('cpu')

# Define the same transformations used during training
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images for ViT
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize using ImageNet stats
])

# Load the test data
test_data_path = "/Users/anishvirkhare/Downloads/archive/test"  # Replace with your test data path
test_data = datasets.ImageFolder(root=test_data_path, transform=transform)

# Create DataLoader for the test data
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Optional: Check test dataset size
print(f"Test samples: {len(test_data)}")

Test samples: 4002


/var/folders/_t/lhc7fyqd4x17cs8p6p_jsyrh0000gn/T/ipykernel_1796/1949572461.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=to

In [8]:
# Initialize metrics
test_loss = 0.0
correct_test = 0
total_test = 0

# Use the same loss function
criterion = nn.CrossEntropyLoss()

# Evaluate the model
model.eval()
with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Testing"):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Track accuracy
        _, predicted = torch.max(outputs, 1)
        correct_test += (predicted == labels).sum().item()
        total_test += labels.size(0)

# Calculate metrics
avg_test_loss = test_loss / len(test_loader)
test_accuracy = 100 * correct_test / total_test

# Print test results
print(f"Test Summary:")
print(f"  Test Loss: {avg_test_loss:.4f}")
print(f"  Test Accuracy: {test_accuracy:.2f}%")

Testing: 100%|████████████████████████████████| 126/126 [05:07<00:00,  2.44s/it]

Test Summary:
  Test Loss: 3.3508
  Test Accuracy: 8.80%


# 8 Classes 

In [3]:
import os

In [4]:
from collections import Counter

# Path to the directory containing the training data
data_dir = "/Users/anishvirkhare/Downloads/archive/train"

# Count images in each class
class_counts = {}
for class_name in os.listdir(data_dir):
    class_path = os.path.join(data_dir, class_name)
    if os.path.isdir(class_path):
        class_counts[class_name] = len([
            file for file in os.listdir(class_path)
            if os.path.isfile(os.path.join(class_path, file))
        ])

# Display the number of images in each class
print("Number of images in each class:")
for class_name, count in class_counts.items():
    print(f"{class_name}: {count}")


Number of images in each class:
Urticaria Hives: 212
Seborrheic Keratoses and other Benign Tumors: 1371
Poison Ivy Photos and other Contact Dermatitis: 260
Acne and Rosacea Photos: 840
Vascular Tumors: 482
Eczema Photos: 1235
Psoriasis pictures Lichen Planus and related diseases: 1405
Exanthems and Drug Eruptions: 404
Lupus and other Connective Tissue diseases: 420
Scabies Lyme Disease and other Infestations and Bites: 431
Bullous Disease Photos: 448
Nail Fungus and other Nail Disease: 1040
Tinea Ringworm Candidiasis and other Fungal Infections: 1300
Systemic Disease: 606
Light Diseases and Disorders of Pigmentation: 568
Atopic Dermatitis Photos: 489
Warts Molluscum and other Viral Infections: 1086
Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions: 1149
Melanoma Skin Cancer Nevi and Moles: 463
Vasculitis Photos: 416
Cellulitis Impetigo and other Bacterial Infections: 288
Hair Loss Photos Alopecia and other Hair Diseases: 239
Herpes HPV and other STDs Photos: 405


In [5]:
import shutil

# Input parameters
data_dir = "/Users/anishvirkhare/Downloads/archive/train"  # Path to the original training data
output_dir = "/Users/anishvirkhare/Downloads/8_classes_train"  # Path to save the filtered training data

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

for class_name in os.listdir(data_dir):
    class_path = os.path.join(data_dir, class_name)
    if os.path.isdir(class_path):
        image_files = [
            file for file in os.listdir(class_path)
            if os.path.isfile(os.path.join(class_path, file))
        ]
        if len(image_files) >= 800:
            # Copy the class directory to the new location
            dest_class_path = os.path.join(output_dir, class_name)
            shutil.copytree(class_path, dest_class_path)
            print(f"Class '{class_name}' retained with {len(image_files)} images.")
        else:
            print(f"Class '{class_name}' removed with only {len(image_files)} images.")

print("Filtered training data saved to:", output_dir)

Class 'Urticaria Hives' removed with only 212 images.
Class 'Seborrheic Keratoses and other Benign Tumors' retained with 1371 images.
Class 'Poison Ivy Photos and other Contact Dermatitis' removed with only 260 images.
Class 'Acne and Rosacea Photos' retained with 840 images.
Class 'Vascular Tumors' removed with only 482 images.
Class 'Eczema Photos' retained with 1235 images.
Class 'Psoriasis pictures Lichen Planus and related diseases' retained with 1405 images.
Class 'Exanthems and Drug Eruptions' removed with only 404 images.
Class 'Lupus and other Connective Tissue diseases' removed with only 420 images.
Class 'Scabies Lyme Disease and other Infestations and Bites' removed with only 431 images.
Class 'Bullous Disease Photos' removed with only 448 images.
Class 'Nail Fungus and other Nail Disease' retained with 1040 images.
Class 'Tinea Ringworm Candidiasis and other Fungal Infections' retained with 1300 images.
Class 'Systemic Disease' removed with only 606 images.
Class 'Light Di

In [6]:
# Define transformations for data augmentation and normalization
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images for ViT
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize using ImageNet stats
])

# Load training data
train_data_path = '/Users/anishvirkhare/Downloads/8_classes_train' 
train_data = datasets.ImageFolder(root=train_data_path, transform=transform)

# Split into training and validation sets
train_size = int(0.9 * len(train_data))
val_size = len(train_data) - train_size
train_dataset, val_dataset = random_split(train_data, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Optional: Check dataset sizes
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

Training samples: 8482
Validation samples: 943


In [7]:
# Create a Vision Transformer model with 23 classes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = create_model('vit_base_patch16_224', pretrained=True, num_classes=23).to(device)

In [8]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=3e-4)

In [10]:
# Training loop
epochs = 10   # Number of epochs
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    # Training
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} - Training"):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track training loss and accuracy
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

    avg_train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train

    # Validation
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} - Validation"):
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Track validation accuracy
            _, predicted = torch.max(outputs, 1)
            correct_val += (predicted == labels).sum().item()
            total_val += labels.size(0)

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * correct_val / total_val

    # Print epoch summary
    print(f"Epoch {epoch+1}/{epochs} Summary:")
    print(f"  Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%")
    print(f"  Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")


Epoch 1/10 - Validation: 100%|██████████████████| 30/30 [02:21<00:00,  4.72s/it]


Epoch 1/10 Summary:
  Training Loss: 2.0283, Training Accuracy: 22.51%
  Validation Loss: 1.8985, Validation Accuracy: 30.01%


Epoch 2/10 - Validation: 100%|██████████████████| 30/30 [01:26<00:00,  2.87s/it]


Epoch 2/10 Summary:
  Training Loss: 1.7976, Training Accuracy: 32.06%
  Validation Loss: 1.9898, Validation Accuracy: 28.31%


Epoch 3/10 - Validation: 100%|██████████████████| 30/30 [01:37<00:00,  3.24s/it]


...

Epoch 8/10 Summary:
  Training Loss: 1.2478, Training Accuracy: 54.75%
  Validation Loss: 1.8452, Validation Accuracy: 39.02%


Epoch 9/10 - Validation: 100%|██████████████████| 30/30 [01:22<00:00,  2.76s/it]


Epoch 9/10 Summary:
  Training Loss: 1.1507, Training Accuracy: 58.21%
  Validation Loss: 1.6159, Validation Accuracy: 43.90%


Epoch 10/10 - Validation: 100%|█████████████████| 30/30 [01:28<00:00,  2.94s/it]

Epoch 10/10 Summary:
  Training Loss: 1.0508, Training Accuracy: 62.54%
  Validation Loss: 1.5618, Validation Accuracy: 46.98%


In [12]:
torch.save(model, "/Users/anishvirkhare/Downloads/final_ViT(8 classes).pth")

In [13]:
model = torch.load("/Users/anishvirkhare/Downloads/final_ViT(8 classes).pth",map_location=torch.device('cpu'))

/var/folders/_t/lhc7fyqd4x17cs8p6p_jsyrh0000gn/T/ipykernel_53892/225716031.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("/Users/anishvirkhare/Downl

In [16]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torch.nn.functional import softmax
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report

In [19]:
# Define the same transformations used during training
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images for ViT
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize using ImageNet stats
])

# Load the test data
test_data_path = "/Users/anishvirkhare/Downloads/8_classes_test"  # Replace with your test data path
test_data = datasets.ImageFolder(root=test_data_path, transform=transform)

# Create DataLoader for the test data
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Optional: Check test dataset size
print(f"Test samples: {len(test_data)}")

Test samples: 2462


In [20]:
# Initialize metrics
test_loss = 0.0
correct_test = 0
total_test = 0

# Use the same loss function
criterion = nn.CrossEntropyLoss()

# Evaluate the model
model.eval()
with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Testing"):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Track accuracy
        _, predicted = torch.max(outputs, 1)
        correct_test += (predicted == labels).sum().item()
        total_test += labels.size(0)

# Calculate metrics
avg_test_loss = test_loss / len(test_loader)
test_accuracy = 100 * correct_test / total_test

# Print test results
print(f"Test Summary:")
print(f"  Test Loss: {avg_test_loss:.4f}")
print(f"  Test Accuracy: {test_accuracy:.2f}%")

Testing: 100%|██████████████████████████████████| 77/77 [02:56<00:00,  2.29s/it]

Test Summary:
  Test Loss: 1.5462
  Test Accuracy: 47.40%


In [9]:
model = torch.load("/Users/anishvirkhare/Downloads/final_ViT(8 classes).pth",map_location=torch.device('cpu'))

/var/folders/_t/lhc7fyqd4x17cs8p6p_jsyrh0000gn/T/ipykernel_1796/225716031.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("/Users/anishvirkhare/Downlo

In [10]:
from sklearn.metrics import classification_report

# Initialize metrics
test_loss = 0.0
correct_test = 0
total_test = 0
all_labels = []
all_predictions = []

# Use the same loss function
criterion = nn.CrossEntropyLoss()

# Evaluate the model
model.eval()
with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Testing"):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Track accuracy
        _, predicted = torch.max(outputs, 1)
        correct_test += (predicted == labels).sum().item()
        total_test += labels.size(0)

        # Collect all labels and predictions for classification report
        all_labels.extend(labels.cpu().numpy())  # Move to CPU and convert to numpy
        all_predictions.extend(predicted.cpu().numpy())

# Calculate metrics
avg_test_loss = test_loss / len(test_loader)
test_accuracy = 100 * correct_test / total_test

# Print test results
print(f"Test Summary:")
print(f"  Test Loss: {avg_test_loss:.4f}")
print(f"  Test Accuracy: {test_accuracy:.2f}%")

# Generate classification report
print("\nClassification Report:")
print(classification_report(all_labels, all_predictions, target_names=test_loader.dataset.classes))


Testing: 100%|████████████████████████████████| 126/126 [04:58<00:00,  2.37s/it]
/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Test Summary:
  Test Loss: 8.9929
  Test Accuracy: 9.67%

Classification Report:
                                                                    precision    recall  f1-score   support

                                           Acne and Rosacea Photos       0.64      0.46      0.54       312
Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions       0.29      0.51      0.37       288
                                          Atopic Dermatitis Photos       0.07      0.31      0.12       123
                                            Bullous Disease Photos       0.01      0.02      0.01       113
                Cellulitis Impetigo and other Bacterial Infections       0.02      0.19      0.03        73
                                                     Eczema Photos       0.02      0.08      0.04       309
                                      Exanthems and Drug Eruptions       0.05      0.18      0.08       101
                 Hair Loss Photos Alopecia and other H

/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
